In [1]:
import pandas as pd
from rdkit import Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Chem import PandasTools
from rdkit import DataStructs
import numpy as np
import time

In [2]:
def extract_template(rxn_smiles,max_prec):
    import rdchiral
    from rdchiral import template_extractor
    templates = []
    for i in index[0:max_prec]:
        rxnmapped = rxn_smiles[i]
        reaction = {
        'reactants': rxnmapped.split('>')[0],
        'products': rxnmapped.split('>')[-1],
        '_id': i,
        }
        template = template_extractor.extract_from_reaction(reaction)
        templates.append(template)
    #print(templates)
    #print(len(templates))
    #display(templates[0]['reaction_smarts'])
    #print('---------------------------')
    rxn_template = []
    n = 0
    for i in templates:
        if i is None or 'reaction_smarts' not in i:
            rxn_smarts = None
            n += 1
            #print(rxn_smarts)
            #print(i['reaction_id'])
            rxn_template.append(rxn_smarts)
            continue
        rxn_smarts = '(' + i['reaction_smarts'].replace('>>', ')>>')
        #print(rxn_smarts)
        #print(i['reaction_id'])
        rxn_template.append(rxn_smarts)
    #print(len(rxn_template))
    #print("number of invalid templates is %d" %n)
    return(rxn_template)

In [3]:
def propose_precursors(rxn_templates):
    from rdkit.Chem import rdChemReactions
    from rdchiral.main import rdchiralReactants, rdchiralReaction, rdchiralRun
    rct = rdchiralReactants(product_smiles)
    rxns = []
    precursors = []
    n=0
    for i in rxn_templates: 
        if i is None:
            print('------------------------------')
            print("invalid rxn_template")
            print('------------------------------')
            rxn = None
            rxns.append(rxn)
            continue
        rxn_i = AllChem.ReactionFromSmarts(i)
        #num_r = rxn_i.GetNumReactantTemplates()
        #num_p = rxn_i.GetNumProductTemplates()
        #n += 1
        #print(num_r, num_p, n)
        #print('------------------------------')
        rxn = rdchiralReaction(i)
        rxns.append(rxn)
        #print(rxns)
    #print('##############################')
    for i in rxns:
        if i is None:
            precursor = None
            precursors.append(precursor)
            # print(precursor)
            continue
        precursor = rdchiralRun(i, rct, combine_enantiomers=False)
        precursors.append(precursor)
        #print(precursor)
    #print(len(precursors))
    return(precursors)

In [4]:
def get_rxns():
    rxns =[]
    coindexs = []
    for idx,i in enumerate(precursors):
        if i is None:
            rxnstr = None
            rxns.append(rxnstr)
            coindex = index[idx]
            coindexs.append(coindex)
            #print('------------------------------')
            #print("this reaction could not propose suitable smiles")
            #print('------------------------------')
            continue
        if len(i) ==0:
            rxnstr = None
            rxns.append(rxnstr)
            coindex = index[idx]
            coindexs.append(coindex)
            #print('------------------------------')
            #print("this reaction could not propose suitable smiles")
            #print('------------------------------')
            continue
        for j in i:
            m = Chem.MolFromSmiles(j,sanitize=False)
            if m is None:
                print('invalid smiles')
                rxnstr = None
                rxns.append(rxnstr)
                coindex = index[idx]
                coindexs.append(coindex)
            else:
                try:
                    Chem.SanitizeMol(m)
                    #print("smiles is ok")
                    #display(m)
                    rxnstr=j +'>>'+product_smiles
                    #print(rxnstr)
                    rxns.append(rxnstr)
                    coindex = index[idx]
                    coindexs.append(coindex)

                except:
                    #print('invalid chemistry')
                    rxnstr = None
                    rxns.append(rxnstr)
                    coindex = index[idx]
                    coindexs.append(coindex) 
    #print(rxns)
    #print(len(rxns))
    #print(coindexs)
    #print(len(coindexs))
    repeat_indexs = []
    t = {}
    for i in range(len(coindexs)):
        for j in range(i):
             if coindexs[i] == coindexs[j]:
                repeat_index = coindexs[j]
                #print(repeat_index)
                repeat_indexs.append(repeat_index)
    #print('------------------------------')
    t = list(zip(coindexs,rxns))
    #print(t)
    #print(len(t))
    return rxns,t

In [5]:
def remove_reactant_smiles():
    Reactant_smiles = []
    Reactant_mols = []
    for rxn_smi in df['Rxn']:
        rcts_smi = rxn_smi.split('>>')[0]
        rcts_mol = Chem.MolFromSmiles(rcts_smi)
        [atom.ClearProp('molAtomMapNumber') for atom in rcts_mol.GetAtoms()]
        Reactant_smile = Chem.MolToSmiles(rcts_mol, True)
        Reactant_smiles.append(Reactant_smile)
        Reactant_mol = Chem.MolFromSmiles(Reactant_smile, True)
        Reactant_mols.append(Reactant_mol) 
    return(Reactant_mols)

In [6]:
def rank():
    s_reacs =[]
    ss=[]
    for x in index:
        for k,v in t:
            if k == x:
                #print(k)

                if v is None:
                    #print(v)
                    s_reac = 0
                    s_reacs.append(s_reac)
                    s = 0
                    ss.append(s)
                    continue
                else:
                    i = str(v).split('>>')[0]
                    #print(i)
                    Proposal_mol = Chem.MolFromSmiles(i)
                    if Proposal_mol is None:
                        s_reac = 0
                        s_reacs.append(s_reac)
                        s = 0
                        ss.append(s)
                        continue
                    mfp_proposal = AllChem.GetMorganFingerprint(Proposal_mol,2,useFeatures=True)
                    mfp_Reactant_mol = AllChem.GetMorganFingerprint(Reactant_mols[x],2,useFeatures=True)
                    s_reac = DataStructs.TanimotoSimilarity(mfp_Reactant_mol, mfp_proposal)
                    s_reacs.append(s_reac)
                    s = s_reac*s_prods[x]
                    ss.append(s)
    #print(ss)
    order = np.argsort(ss)[::-1]
    #print(order)
    #print(len(ss))
    return(order)

In [7]:
def remove_atom_mapping():
    df_1=pd.read_csv("data_test.csv",encoding="gbk")
    remove_prod_smiles = []
    remove_rcts_smiles = []
    remove_rxns_smiles = []

    for rxn_smi in df_1['Rxn']:
        prod_smi = rxn_smi.split('>>')[-1]
        prod_mol = Chem.MolFromSmiles(prod_smi)
        [atom.ClearProp('molAtomMapNumber') for atom in prod_mol.GetAtoms()]
        prod_smi_remove_atom_map = Chem.MolToSmiles(prod_mol, True)
        prod_smi_remove_atom_map = Chem.MolToSmiles(Chem.MolFromSmiles(prod_smi_remove_atom_map), True)
        remove_prod_smiles.append(prod_smi_remove_atom_map)      
    for rxn_smi in df_1['Rxn']:
        rcts_smi = rxn_smi.split('>>')[0]
        rcts_mol = Chem.MolFromSmiles(rcts_smi)
        [atom.ClearProp('molAtomMapNumber') for atom in rcts_mol.GetAtoms()]
        rcts_smi_remove_atom_map = Chem.MolToSmiles(rcts_mol, True)
        # Sometimes stereochem takes another canonicalization...
        rcts_smi_remove_atom_map = Chem.MolToSmiles(Chem.MolFromSmiles(rcts_smi_remove_atom_map), True)
        remove_rcts_smiles.append(rcts_smi_remove_atom_map)            

        rxn_smi_remove_atom_map = rcts_smi_remove_atom_map + '>>' + prod_smi_remove_atom_map
        remove_rxns_smiles.append(rxn_smi_remove_atom_map) 
    #print(remove_prod_smiles)
    #print('***********************')
    #print(remove_rcts_smiles)
    #print('***********************')
    #print(remove_rxns_smiles)    
    dataframe_test_remove = pd.DataFrame({'Product':remove_prod_smiles[:],'Reactant':remove_rcts_smiles[:],'Rxn':remove_rxns_smiles[:]})
    dataframe_test_remove.to_csv('remove_data_test.csv',index=False)
    return()

In [8]:
def get_accuracy(n):
    accuracy = []
    m = 0
    for i,j in enumerate(order):
        if i < n:
            if rxns[j] is None:
                continue
            x = rxns[j].split('>>')[0]
            if x == true_precursors:
                a = np.zeros(i,dtype = int)
                b = np.ones(n-i,dtype = int)
                accuracy = np.concatenate((a,b))
                #print(accuracy)
                m = m +1
                #print(m)
                break
        else:
            break
    if m == 0:
        accuracy = np.zeros(n,dtype = int)
        #print(accuracy)
    return(accuracy)

In [9]:
#prepare_data
data=pd.read_csv("data_processed.csv",encoding="gbk")
#data=pd.read_csv("炸药合成.csv",encoding="gbk")
dataframe = pd.DataFrame({'Product':data['prod_smiles'][:45000],'Rxn':data['rxn_smiles'][:45000]})
dataframe.to_csv('data_train.csv',index=False)
dataframe_test = pd.DataFrame({'Product':data['prod_smiles'][45000:50000],'Rxn':data['rxn_smiles'][45000:50000]})
dataframe_test.to_csv('data_test.csv',index=False)

In [10]:
print('Starting......')
start_time = time.time()
#train:smiles2mol2fingerprint
Product_mols = []
df=pd.read_csv("data_train.csv",encoding="gbk")
for i in df['Product']:
    Product_mol = Chem.MolFromSmiles(i)
    Product_mols.append(Product_mol)
mfps = []
for i in Product_mols:
    mfp = AllChem.GetMorganFingerprint(i,2,useFeatures=True)
    mfps.append(mfp)

Starting......


In [11]:
#test:smiles2mol2fingerprint
df_1=pd.read_csv("data_test.csv",encoding="gbk")
#df_1=pd.read_csv("炸药合成.csv",encoding="gbk")
#product_smiles = 'O=[N+]([O-])NC1=NN=C(N[N+](=O)[O-])N=N1'
#product_smiles = 'O=[N+]([O-])N1CN([N+](=O)[O-])CN([N+](=O)[O-])CN([N+](=O)[O-])C1'
#product_smiles = 'CC(=O)N1CN(C(C)=O)CN([N+](=O)[O-])CN(C(C)=O)C1'
#product_smiles = 'Cc1c(-c2ccncc2)nc2cc(F)cc(F)c2c1Nc1cc(N2CCOCC2)cnc1N1CCOCC1'
Reactant_mols = remove_reactant_smiles()
remove_atom_mapping()
n = 1
accuracies = np.empty([0,n])
total_length = len(df_1)
print('total length: ', total_length)

for a,i in enumerate(df_1['Product']):
    curr_time = time.time()
    spent_time = curr_time - start_time
    print('(', a+1, ' / ', total_length, ')......', "Spent time:", spent_time)
    start_time = time.time()
    
    product_smiles = i
    product_mol = Chem.MolFromSmiles(product_smiles)
    product_mfp = AllChem.GetMorganFingerprint(product_mol,2,useFeatures=True)
    #get product_similarity
    s_prods = []
    for i in mfps:
        s_prod = DataStructs.TanimotoSimilarity(product_mfp,i)
        s_prods.append(s_prod)
    #print(s_prods)
    index = np.argsort(s_prods)[::-1]
    #print(index)
    #print(len(index))
    rxn_template = extract_template(data['rxn_smiles'],200)
    precursors = propose_precursors(rxn_template)
    rxns,t = get_rxns()
    order = rank()
    df_2 = pd.read_csv("remove_data_test.csv",encoding="gbk")
    true_precursors = df_2['Reactant'][a]
    #true_precursors = df_1['Reactant'][a]
    accuracy = get_accuracy(n)
    accuracies = np.append(accuracies,[accuracy],axis=0)
    if a % int(total_length//20) == 0:
        print(5*a//(total_length//20), "%, ", 'accuracy:', np.mean(accuracies, axis=0))
#print(accuracies)
mean_accuracies = np.mean(accuracies, axis=0)
print(f"Top {n} accuracy: {mean_accuracies[n-1] * 100: .2f} %")

total length:  5000
( 1  /  5000 )...... Spent time: 82.23022365570068
0 %,  accuracy: [0.]
( 2  /  5000 )...... Spent time: 3.592679977416992
( 3  /  5000 )...... Spent time: 2.9092605113983154
( 4  /  5000 )...... Spent time: 2.355302333831787
( 5  /  5000 )...... Spent time: 3.4532322883605957
( 6  /  5000 )...... Spent time: 2.607111692428589
( 7  /  5000 )...... Spent time: 2.7011821269989014
( 8  /  5000 )...... Spent time: 3.2880613803863525
( 9  /  5000 )...... Spent time: 2.8579986095428467
( 10  /  5000 )...... Spent time: 3.0930519104003906
( 11  /  5000 )...... Spent time: 3.22395658493042
( 12  /  5000 )...... Spent time: 3.9511144161224365
( 13  /  5000 )...... Spent time: 3.1840484142303467
( 14  /  5000 )...... Spent time: 3.284266471862793
( 15  /  5000 )...... Spent time: 2.6569557189941406
( 16  /  5000 )...... Spent time: 4.7518470287323
( 17  /  5000 )...... Spent time: 3.4096946716308594
( 18  /  5000 )...... Spent time: 2.8299708366394043
( 19  /  5000 )...... Sp

( 156  /  5000 )...... Spent time: 3.281038761138916
( 157  /  5000 )...... Spent time: 2.925015926361084
( 158  /  5000 )...... Spent time: 3.059133768081665
( 159  /  5000 )...... Spent time: 3.8547706604003906
( 160  /  5000 )...... Spent time: 2.4981775283813477
( 161  /  5000 )...... Spent time: 2.9350976943969727
( 162  /  5000 )...... Spent time: 2.3405728340148926
( 163  /  5000 )...... Spent time: 2.2985076904296875
( 164  /  5000 )...... Spent time: 2.659130334854126
( 165  /  5000 )...... Spent time: 2.7294702529907227
( 166  /  5000 )...... Spent time: 3.2819950580596924
( 167  /  5000 )...... Spent time: 3.380002498626709
( 168  /  5000 )...... Spent time: 2.8529698848724365
( 169  /  5000 )...... Spent time: 3.503549575805664
( 170  /  5000 )...... Spent time: 3.31103515625
( 171  /  5000 )...... Spent time: 2.786980390548706
( 172  /  5000 )...... Spent time: 3.2920005321502686
( 173  /  5000 )...... Spent time: 3.5995147228240967
( 174  /  5000 )...... Spent time: 1.914

( 309  /  5000 )...... Spent time: 2.294990301132202
( 310  /  5000 )...... Spent time: 3.889965295791626
( 311  /  5000 )...... Spent time: 2.7330169677734375
( 312  /  5000 )...... Spent time: 2.1759753227233887
( 313  /  5000 )...... Spent time: 3.190678596496582
( 314  /  5000 )...... Spent time: 2.7610907554626465
( 315  /  5000 )...... Spent time: 2.0731770992279053
( 316  /  5000 )...... Spent time: 4.100713729858398
( 317  /  5000 )...... Spent time: 2.9736483097076416
( 318  /  5000 )...... Spent time: 2.2321407794952393
( 319  /  5000 )...... Spent time: 3.194636344909668
( 320  /  5000 )...... Spent time: 2.803513288497925
( 321  /  5000 )...... Spent time: 3.531203031539917
( 322  /  5000 )...... Spent time: 3.9590749740600586
( 323  /  5000 )...... Spent time: 2.486815929412842
( 324  /  5000 )...... Spent time: 3.5678811073303223
( 325  /  5000 )...... Spent time: 2.1170225143432617
( 326  /  5000 )...... Spent time: 4.528940677642822
( 327  /  5000 )...... Spent time: 4.

( 463  /  5000 )...... Spent time: 2.2019646167755127
( 464  /  5000 )...... Spent time: 2.3939967155456543
( 465  /  5000 )...... Spent time: 2.7670037746429443
( 466  /  5000 )...... Spent time: 2.2043373584747314
( 467  /  5000 )...... Spent time: 2.7274227142333984
( 468  /  5000 )...... Spent time: 3.6749300956726074
( 469  /  5000 )...... Spent time: 2.07295298576355
( 470  /  5000 )...... Spent time: 3.021577835083008
( 471  /  5000 )...... Spent time: 3.007614850997925
( 472  /  5000 )...... Spent time: 2.032132387161255
( 473  /  5000 )...... Spent time: 3.015568733215332
( 474  /  5000 )...... Spent time: 3.154005289077759
( 475  /  5000 )...... Spent time: 2.614150047302246
( 476  /  5000 )...... Spent time: 2.7865073680877686
( 477  /  5000 )...... Spent time: 2.5712780952453613
( 478  /  5000 )...... Spent time: 3.3357996940612793
( 479  /  5000 )...... Spent time: 3.8241066932678223
( 480  /  5000 )...... Spent time: 2.8299129009246826
( 481  /  5000 )...... Spent time: 4

( 616  /  5000 )...... Spent time: 3.7176337242126465
( 617  /  5000 )...... Spent time: 2.779963970184326
( 618  /  5000 )...... Spent time: 4.072784185409546
( 619  /  5000 )...... Spent time: 2.5459372997283936
( 620  /  5000 )...... Spent time: 3.9839189052581787
( 621  /  5000 )...... Spent time: 2.761451244354248
( 622  /  5000 )...... Spent time: 2.145463705062866
( 623  /  5000 )...... Spent time: 3.463742971420288
( 624  /  5000 )...... Spent time: 3.8429994583129883
( 625  /  5000 )...... Spent time: 3.1866674423217773
( 626  /  5000 )...... Spent time: 4.345766067504883
( 627  /  5000 )...... Spent time: 2.300189971923828
( 628  /  5000 )...... Spent time: 2.72200083732605
( 629  /  5000 )...... Spent time: 2.4380240440368652
( 630  /  5000 )...... Spent time: 3.82498836517334
( 631  /  5000 )...... Spent time: 2.455575942993164
( 632  /  5000 )...... Spent time: 3.610393524169922
( 633  /  5000 )...... Spent time: 3.385594367980957
( 634  /  5000 )...... Spent time: 3.36801

...

( 4386  /  5000 )...... Spent time: 3.329272508621216
( 4387  /  5000 )...... Spent time: 3.5777783393859863
( 4388  /  5000 )...... Spent time: 2.348977565765381
( 4389  /  5000 )...... Spent time: 2.6688477993011475
( 4390  /  5000 )...... Spent time: 3.115534543991089
( 4391  /  5000 )...... Spent time: 2.3159124851226807
( 4392  /  5000 )...... Spent time: 4.204667568206787
( 4393  /  5000 )...... Spent time: 2.0032246112823486
( 4394  /  5000 )...... Spent time: 2.282297134399414
( 4395  /  5000 )...... Spent time: 3.1684961318969727
( 4396  /  5000 )...... Spent time: 2.2508292198181152
( 4397  /  5000 )...... Spent time: 3.9482038021087646
( 4398  /  5000 )...... Spent time: 2.64894962310791
( 4399  /  5000 )...... Spent time: 3.4420223236083984
( 4400  /  5000 )...... Spent time: 3.3909988403320312
( 4401  /  5000 )...... Spent time: 2.2650012969970703
( 4402  /  5000 )...... Spent time: 3.519094228744507
( 4403  /  5000 )...... Spent time: 3.0730299949645996
( 4404  /  5000 ).

( 4536  /  5000 )...... Spent time: 3.8571667671203613
( 4537  /  5000 )...... Spent time: 2.6390128135681152
( 4538  /  5000 )...... Spent time: 3.842000722885132
( 4539  /  5000 )...... Spent time: 3.554152250289917
( 4540  /  5000 )...... Spent time: 4.197232007980347
( 4541  /  5000 )...... Spent time: 2.1640639305114746
( 4542  /  5000 )...... Spent time: 3.7659871578216553
( 4543  /  5000 )...... Spent time: 2.878383159637451
( 4544  /  5000 )...... Spent time: 3.8193068504333496
( 4545  /  5000 )...... Spent time: 3.9056100845336914
( 4546  /  5000 )...... Spent time: 3.128608465194702
( 4547  /  5000 )...... Spent time: 3.4174351692199707
( 4548  /  5000 )...... Spent time: 2.480254888534546
( 4549  /  5000 )...... Spent time: 4.708765029907227
( 4550  /  5000 )...... Spent time: 3.8204665184020996
( 4551  /  5000 )...... Spent time: 2.15472674369812
( 4552  /  5000 )...... Spent time: 4.826398849487305
( 4553  /  5000 )...... Spent time: 4.1355979442596436
( 4554  /  5000 )...

( 4687  /  5000 )...... Spent time: 3.8320112228393555
( 4688  /  5000 )...... Spent time: 4.3949668407440186
( 4689  /  5000 )...... Spent time: 3.6510331630706787
( 4690  /  5000 )...... Spent time: 4.308099031448364
( 4691  /  5000 )...... Spent time: 3.0970327854156494
( 4692  /  5000 )...... Spent time: 2.50896954536438
( 4693  /  5000 )...... Spent time: 2.4897851943969727
( 4694  /  5000 )...... Spent time: 3.7532875537872314
( 4695  /  5000 )...... Spent time: 3.1684277057647705
( 4696  /  5000 )...... Spent time: 3.057194232940674
( 4697  /  5000 )...... Spent time: 2.1851513385772705
( 4698  /  5000 )...... Spent time: 4.424389839172363
( 4699  /  5000 )...... Spent time: 3.999610185623169
( 4700  /  5000 )...... Spent time: 3.7278947830200195
( 4701  /  5000 )...... Spent time: 2.3110413551330566
( 4702  /  5000 )...... Spent time: 3.308959484100342
( 4703  /  5000 )...... Spent time: 2.6940033435821533
( 4704  /  5000 )...... Spent time: 3.182995557785034
( 4705  /  5000 ).

( 4837  /  5000 )...... Spent time: 2.8317112922668457
( 4838  /  5000 )...... Spent time: 2.258958339691162
( 4839  /  5000 )...... Spent time: 3.446038007736206
( 4840  /  5000 )...... Spent time: 2.528862953186035
( 4841  /  5000 )...... Spent time: 3.614208936691284
( 4842  /  5000 )...... Spent time: 3.911470413208008
( 4843  /  5000 )...... Spent time: 3.96368408203125
( 4844  /  5000 )...... Spent time: 3.545705795288086
( 4845  /  5000 )...... Spent time: 3.9597818851470947
( 4846  /  5000 )...... Spent time: 3.8037734031677246
( 4847  /  5000 )...... Spent time: 2.311976909637451
( 4848  /  5000 )...... Spent time: 2.847010374069214
( 4849  /  5000 )...... Spent time: 2.1755754947662354
( 4850  /  5000 )...... Spent time: 3.1250040531158447
( 4851  /  5000 )...... Spent time: 2.643000602722168
( 4852  /  5000 )...... Spent time: 2.211033582687378
( 4853  /  5000 )...... Spent time: 3.0979678630828857
( 4854  /  5000 )...... Spent time: 2.517059087753296
( 4855  /  5000 )......

( 4988  /  5000 )...... Spent time: 2.9360055923461914
( 4989  /  5000 )...... Spent time: 2.1109933853149414
( 4990  /  5000 )...... Spent time: 4.395971775054932
( 4991  /  5000 )...... Spent time: 2.6770331859588623
( 4992  /  5000 )...... Spent time: 2.5789706707000732
( 4993  /  5000 )...... Spent time: 2.8500335216522217
( 4994  /  5000 )...... Spent time: 3.7050023078918457
( 4995  /  5000 )...... Spent time: 2.936995267868042
( 4996  /  5000 )...... Spent time: 2.820963144302368
( 4997  /  5000 )...... Spent time: 1.899346113204956
( 4998  /  5000 )...... Spent time: 2.7849998474121094
( 4999  /  5000 )...... Spent time: 2.6489999294281006
( 5000  /  5000 )...... Spent time: 3.336001396179199
Top 1 accuracy:  39.74 %
